In [1]:
import re # We will import the 're' module for regular expression operations
from typing import List, Tuple, Optional, Dict, Any # These are for better code clarity and type checking
from dataclasses import dataclass # We have imported this to create classes that automatically generate special methods.

# Types of tokens that will be accepted in our code
class TokenType:
    NUMBER = 'NUMBER'         # Numbers
    IDENTIFIER = 'IDENTIFIER' # Identifiers like variable names
    ASSIGN = 'ASSIGN'         # Assignment operator '='
    PLUS = 'PLUS'             # Plus operator '+'
    MINUS = 'MINUS'           # Minus operator '-'
    MULTIPLY = 'MULTIPLY'     # Multiplication operator '*'
    DIVIDE = 'DIVIDE'         # Division operator '/'
    LPAREN = 'LPAREN'         # Left parenthesis '('
    RPAREN = 'RPAREN'         # Right parenthesis ')'
    SEMICOLON = 'SEMICOLON'   # Semicolon ';'
    PRINT = 'PRINT'           # Print keyword
    IF = 'IF'                 # IF keyword
    ELSE = 'ELSE'             # ELSE keyword
    WHILE = 'WHILE'           # WHILE keyword
    DO = 'DO'                 # DO keyword
    LT = 'LT'                 # Less than '<'
    GT = 'GT'                 # Greater than '>'
    LTE = 'LTE'               # Less than or equal '<='
    GTE = 'GTE'               # Greater than or equal '>='
    EQ = 'EQ'                 # Equal '=='
    NEQ = 'NEQ'               # Not equal '!='
    OP = 'OP'                 # Operators '+', '-', '*', '/'
    REL_OP = 'REL_OP'         # Relational operators '==', '!=', '<=', '>='
    LBRACE = 'LBRACE'         # Left brace '{'
    RBRACE = 'RBRACE'         # Right brace '}'
    SKIP = 'SKIP'             # Spaces, tabs, newlines
    MISMATCH = 'MISMATCH'     # Mismatched characters

# The specifications for each token in the code
TOKEN_SPECIFICATION = [
    (TokenType.NUMBER, r'\d+(\.\d*)?'),                  # Numbers
    (TokenType.IF, r'\bif\b'),                           # IF keyword
    (TokenType.ELSE, r'\belse\b'),                       # ELSE keyword
    (TokenType.WHILE, r'\bwhile\b'),                     # WHILE keyword
    (TokenType.DO, r'\bdo\b'),                           # DO keyword
    (TokenType.PRINT, r'\bprint\b'),                     # PRINT keyword
    (TokenType.IDENTIFIER, r'[A-Za-z_][A-Za-z0-9_]*'),   # Identifiers
    (TokenType.ASSIGN, r'='),                            # Assignment operator
    (TokenType.SEMICOLON, r';'),                         # Semicolon
    (TokenType.OP, r'[+\-*/]'),                          # Operators
    (TokenType.REL_OP, r'==|!=|<=|>=|<|>'),              # Relational operators
    (TokenType.LPAREN, r'\('),                           # Left parenthesis
    (TokenType.RPAREN, r'\)'),                           # Right parenthesis
    (TokenType.LBRACE, r'\{'),                           # Left brace
    (TokenType.RBRACE, r'\}'),                           # Right brace
    (TokenType.SKIP, r'[ \t\n]+'),                       # Skip spaces, tabs, newlines
    (TokenType.MISMATCH, r'.'),                          # Any other character
]

# Custom Exceptions
class LexicalError(Exception):
    pass

class SyntaxError(Exception):
    pass

# PROCEDURAL
# The code for our Lexical Analysis part
# We will get each characters and turn them into tokens 
def tokenize(code: str) -> list[tuple[str, str]]:
    tokens = []
    token_regex = '|'.join(f'(?P<{name}>{pattern})' for name, pattern in TOKEN_SPECIFICATION)
    position = 0

    print("\nLexical Analysis:")
    print("-" * 50)
    print("| Token Type    | Token Value |")
    print("-" * 50)

    while position < len(code):
        match = re.match(token_regex, code[position:])
        if not match:
            raise LexicalError(f"Illegal character '{code[position]}' at position {position}")

        kind = match.lastgroup
        value = match.group()

        if kind == 'SKIP':
            position += len(value)
            continue

        tokens.append((kind, value))
        print(f"| {kind:<12} | {value:^11} |")
        position += len(value)

    print("-" * 50 + "\n")
    return tokens

# OBJECT-ORIENTED
# Second part is the Syntax Analysis
class Parser:
    # We have initialized the parser with a list of tokens
    # The current position is set to 0
    def __init__(self, tokens: List[Tuple[str, str]]):
        self.tokens = tokens
        self.pos = 0
        print("Syntax Analysis:")
        print("-" * 50)
        print("Parsed value:")
    
    # If at the end of tokens, it will return the current token without moving forward or None
    def peek(self) -> Optional[Tuple[str, str]]:
        return self.tokens[self.pos] if self.pos < len(self.tokens) else None
    
    # This will check each token if it matches our expected type and will move to the next token and return the current token
    # Will raise an error if the type is not as expected
    def consume(self, expected_type: str) -> Tuple[str, str]:
        token = self.peek()
        if not token:
            raise SyntaxError(f"Unexpected end of input, expected {expected_type}")
        if token[0] != expected_type:
            raise SyntaxError(f"Expected {expected_type} but got {token[0]}")
        
        self.pos += 1
        return token
    
    # Prints the AST in a dictionary-like format
    def print_ast_dict(self, node, indent=0):
        if isinstance(node, tuple):
            print(" " * indent + f"type: {node[0]}")
            
            if node[0] == 'BinOp':
                print(" " * indent + f"operator: {node[1]}")
                print(" " * indent + "left:")
                self.print_ast_dict(node[2], indent + 2)
                print(" " * indent + "right:")
                self.print_ast_dict(node[3], indent + 2)
            
            elif node[0] == 'RelOp':
                print(" " * indent + f"operator: {node[1]}")
                print(" " * indent + "left:")
                self.print_ast_dict(node[2], indent + 2)
                print(" " * indent + "right:")
                self.print_ast_dict(node[3], indent + 2)
            
            elif node[0] == 'Program':
                print(" " * indent + "statements:")
                self.print_ast_dict(node[1], indent + 2)
            
            elif node[0] == 'Assign':
                print(" " * indent + f"variable: {node[1]}")
                print(" " * indent + "value:")
                self.print_ast_dict(node[2], indent + 2)
            
            elif node[0] == 'Print':
                print(" " * indent + "expression:")
                self.print_ast_dict(node[1], indent + 2)
            
            elif node[0] == 'If':
                print(" " * indent + "condition:")
                self.print_ast_dict(node[1], indent + 2)
                print(" " * indent + "then_block:")
                self.print_ast_dict(node[2], indent + 2)
                if node[3]:  # else block exists
                    print(" " * indent + "else_block:")
                    self.print_ast_dict(node[3], indent + 2)
            
            elif node[0] == 'While':
                print(" " * indent + "condition:")
                self.print_ast_dict(node[1], indent + 2)
                print(" " * indent + "body:")
                self.print_ast_dict(node[2], indent + 2)
            
            elif node[0] == 'DoWhile':
                print(" " * indent + "body:")
                self.print_ast_dict(node[2], indent + 2)
                print(" " * indent + "condition:")
                self.print_ast_dict(node[1], indent + 2)
            
            elif node[0] == 'Number':
                print(" " * indent + f"value: {node[1]}")
            
            elif node[0] == 'Variable':
                print(" " * indent + f"name: {node[1]}")
        
        elif isinstance(node, list):
            for item in node:
                self.print_ast_dict(item, indent + 2)
        else:
            print(" " * indent + str(node))

    def parse(self):
        statements = []
        while self.peek():
            statements.append(self.parse_statement())
        return ('Program', statements)
    
    # For arithmetic expressions that may include relational operators
    def parse_expression(self):
        left = self.parse_arithmetic()
        
        while self.peek() and self.peek()[0] == TokenType.REL_OP:
            op = self.consume(TokenType.REL_OP)
            right = self.parse_arithmetic()
            left = ('RelOp', op[1], left, right)
        
        return left
    
    # For arithmetic expressions that may include binary operations
    def parse_arithmetic(self):
        node = self.parse_term()
        
        while self.peek() and self.peek()[0] == TokenType.OP:
            op = self.consume(TokenType.OP)
            right = self.parse_term()
            node = ('BinOp', op[1], node, right)
        
        return node
    
    # For a term that could be an expression in parentheses, a number or a variable
    def parse_term(self):
        token = self.peek()
        
        if not token:
            raise SyntaxError("Unexpected end of input in term")
        
        if token[0] == TokenType.LPAREN:
            self.consume(TokenType.LPAREN)
            expr = self.parse_expression()
            self.consume(TokenType.RPAREN)
            return expr
        elif token[0] == TokenType.IDENTIFIER:
            id_token = self.consume(TokenType.IDENTIFIER)
            if self.peek() and self.peek()[0] == TokenType.LPAREN:
                self.consume(TokenType.LPAREN)
                args = self.parse_expression()
                self.consume(TokenType.RPAREN)
                return ('FunctionCall', id_token[1], args)
            return ('Variable', id_token[1])
        elif token[0] == TokenType.NUMBER:
            num = self.consume(TokenType.NUMBER)
            return ('Number', num[1])
        else:
            raise SyntaxError(f"Unexpected token in term: {token}")
    
    # For statements that could be a while, if-else, do-while, print or assignment
    def parse_statement(self):
        token = self.peek()

        if not token:
            raise SyntaxError("Unexpected end of input in statement")

        if token[0] == TokenType.IF:
            return self.parse_if_statement()
        elif token[0] == TokenType.WHILE:
            return self.parse_while_statement()
        elif token[0] == TokenType.DO:
            return self.parse_do_while_statement()
        elif token[0] == TokenType.PRINT:
            return self.parse_print_statement()
        elif token[0] == TokenType.IDENTIFIER:
            return self.parse_assignment()
        else:
            raise SyntaxError(f"Unknown statement type: {token}")

    # Parsing do-while statement
    def parse_do_while_statement(self):
        self.consume(TokenType.DO)
        self.consume(TokenType.LBRACE)
        block = self.parse_block()
        self.consume(TokenType.RBRACE)
        self.consume(TokenType.WHILE)
        self.consume(TokenType.LPAREN)
        condition = self.parse_expression()
        self.consume(TokenType.RPAREN)
        self.consume(TokenType.SEMICOLON)
        return ('DoWhile', condition, block)

    # Parsing while statement
    def parse_while_statement(self):
        self.consume(TokenType.WHILE)
        self.consume(TokenType.LPAREN)
        condition = self.parse_expression()
        self.consume(TokenType.RPAREN)
        self.consume(TokenType.LBRACE)
        block = self.parse_block()
        self.consume(TokenType.RBRACE)
        return ('While', condition, block)

    # Parsing if-else statement
    def parse_if_statement(self):
        self.consume(TokenType.IF)
        self.consume(TokenType.LPAREN)
        condition = self.parse_expression()
        self.consume(TokenType.RPAREN)
        
        self.consume(TokenType.LBRACE)
        if_block = self.parse_block()
        self.consume(TokenType.RBRACE)
        
        else_block = None
        if self.peek() and self.peek()[0] == TokenType.ELSE:
            self.consume(TokenType.ELSE)
            self.consume(TokenType.LBRACE)
            else_block = self.parse_block()
            self.consume(TokenType.RBRACE)
        
        return ('If', condition, if_block, else_block)
    
    # Parsing print statement for the output
    def parse_print_statement(self):
        self.consume(TokenType.PRINT)
        self.consume(TokenType.LPAREN)
        expr = self.parse_expression()
        self.consume(TokenType.RPAREN)
        self.consume(TokenType.SEMICOLON)
        return ('Print', expr)
    
    # Parsing assignment statement to assign a value to a variable
    def parse_assignment(self):
        var = self.consume(TokenType.IDENTIFIER)
        self.consume(TokenType.ASSIGN)
        expr = self.parse_expression()
        self.consume(TokenType.SEMICOLON)
        return ('Assign', var[1], expr)
    
    # Parsing a block of statements until a closing brace is found
    def parse_block(self):
        statements = []
        while self.peek() and self.peek()[0] != TokenType.RBRACE:
            statements.append(self.parse_statement())
        return statements

# This is the Environment and Interpreter part
class Environment:
    def __init__(self, parent=None):
        self.values: Dict[str, Any] = {} # Dictionary to store the variable names and their values
        self.parent = parent
    
    # Defining a new varible in the current environment
    def define(self, name: str, value: Any) -> None:
        self.values[name] = value
    
    # To get the value of a variable from the environment
    def get(self, name: str) -> Any:
        if name in self.values:
            return self.values[name]
        if self.parent:
            return self.parent.get(name)
        raise RuntimeError(f"Variable '{name}' is not defined")
    
    # To assign a new value to an existing variable
    def assign(self, name: str, value: Any) -> None:
        if name in self.values:
            self.values[name] = value
        elif self.parent:
            self.parent.assign(name, value)
        else:
            raise RuntimeError(f"Variable '{name}' is not defined")

class Interpreter:
    def __init__(self):
        self.environment = Environment()
    
    # Evaluating the given AST node
    def evaluate(self, node) -> Any:
        if isinstance(node, tuple):
            node_type = node[0]

            # Executes the loop while the condition is true
            if node_type == 'While':
                while self.evaluate(node[1]):
                    self.evaluate_statements(node[2])
            
            # Executes the code at least once, then checks the condition
            elif node_type == 'DoWhile':
                while True:
                    self.evaluate_statements(node[2])
                    if not self.evaluate(node[1]):
                        break
            
            elif node_type == 'Program':
                return self.evaluate_statements(node[1])
            
            # Returns the number as float or int
            elif node_type == 'Number':
                value = node[1]
                return float(value) if '.' in value else int(value)
            
            # This will retrieve the variable value
            elif node_type == 'Variable':
                return self.environment.get(node[1])
            
            # Evaluates binary operation
            elif node_type == 'BinOp':
                left = self.evaluate(node[2])
                right = self.evaluate(node[3])
                op = node[1]
                
                if op == '+': # Addition
                    return left + right
                elif op == '-': # Subtraction
                    return left - right
                elif op == '*': # Multiplication
                    return left * right
                elif op == '/': # Division
                    if right == 0:
                        raise RuntimeError("Division by zero")
                    return left / right
            
            # For relational operations
            elif node_type == 'RelOp':
                left = self.evaluate(node[2])
                right = self.evaluate(node[3])
                op = node[1]
                
                if op == '>': # More than
                    return left > right 
                elif op == '<': # Less than
                    return left < right
                elif op == '>=': # More than or equals to
                    return left >= right
                elif op == '<=': # Less than or equals to
                    return left <= right
                elif op == '==': # Equality check
                    return left == right
                elif op == '!=': # Inequality check
                    return left != right
            
            # Assign a value to a variable
            elif node_type == 'Assign':
                value = self.evaluate(node[2])
                if isinstance(value, (int, float)):
                    self.environment.define(node[1], value)
                else:
                    raise RuntimeError("Only int and float types are supported for assignment.")
            
            # Evaluates the if-else condition and executes the correct code
            elif node_type == 'If':
                condition = self.evaluate(node[1])
                if condition:
                    return self.evaluate_statements(node[2])
                elif node[3]:  # else block exists
                    return self.evaluate_statements(node[3])
            
            # Prints the evaluated expression
            elif node_type == 'Print':
                value = self.evaluate(node[1])
                print(f"Output: {value}")
                return value
            
            else:
                raise RuntimeError(f"Unknown node type: {node_type}")
        
        return node
    
    def evaluate_statements(self, statements: List) -> Any:
        result = None
        for statement in statements:
            result = self.evaluate(statement)
        return result

class Compiler:
    def __init__(self):
        self.interpreter = Interpreter()
    
    def compile_and_run(self, source_code: str) -> None:
        try:
            print("=" * 50)
            print("COMPILATION PROCESS")
            print("=" * 50)
            
            tokens = tokenize(source_code)  # This will tokenize the source code into smaller manageable pieces
            parser = Parser(tokens)  # This is to create a parser instance with the tokens
            ast = parser.parse() # This code is to parse the tokens to produce an abstract syntax tree (AST)

            print("\nAbstract Syntax Tree:")
            parser.print_ast_dict(ast)  # Print the AST with the node argument
                    
            # Interpretation/Execution of the AST
            print("\nProgram Output:")
            print("-" * 20)  # Prepare to show the program's output
            self.interpreter.evaluate(ast) # Evaluate the AST to execute the program
            
            print("\nCompilation completed successfully.")
            print("=" * 50)
            
        # to handle any errors that arise during compilation process or execution
        except (LexicalError, SyntaxError, RuntimeError) as e:
            print("\nError during compilation:")
            print(f"  {str(e)}")
            print("=" * 50)

# This is to run the code in an Interactive Loop where the users can enter the source code and the compiler will process
def interactive_mode():
    compiler = Compiler()
    print("Welcome to the MiniLang Compiler!!!")
    print("Please enter your code below. Type 'exit' to quit.Thank you.\n")
    print("-" * 50)

    while True:
        try:
            # Get user input for source code
            source_code = input(">>> ")
            if source_code.strip().lower() == "exit":
                print("Exiting...")
                break

            # Compile and run the user-provided code
            compiler.compile_and_run(source_code)
            
        except KeyboardInterrupt:
            print("\nCompilation interrupted.")
            break
        except Exception as e:
            print(f"\nUnexpected error: {str(e)}")

if __name__ == "__main__":
    interactive_mode()

Welcome to the MiniLang Compiler!!!
Please enter your code below. Type 'exit' to quit.Thank you.

--------------------------------------------------
>>> x = 0; while (x < 5) {     print(x);     x = x + 1; }
COMPILATION PROCESS

Lexical Analysis:
--------------------------------------------------
| Token Type    | Token Value |
--------------------------------------------------
| IDENTIFIER   |      x      |
| ASSIGN       |      =      |
| NUMBER       |      0      |
| SEMICOLON    |      ;      |
| WHILE        |    while    |
| LPAREN       |      (      |
| IDENTIFIER   |      x      |
| REL_OP       |      <      |
| NUMBER       |      5      |
| RPAREN       |      )      |
| LBRACE       |      {      |
| PRINT        |    print    |
| LPAREN       |      (      |
| IDENTIFIER   |      x      |
| RPAREN       |      )      |
| SEMICOLON    |      ;      |
| IDENTIFIER   |      x      |
| ASSIGN       |      =      |
| IDENTIFIER   |      x      |
| OP           |      +      |
